In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!sudo pip install mtcnn
!wget 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip'
!unzip ngrok-stable-windows-amd64.zip
!ngrok authtoken 1ptGlHPRRq0SOuf93xUijgxjHoh_3oVnNMTu52qvyLw9aL2vs

--2021-03-18 08:07:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.37.54, 54.145.36.98, 3.223.239.191, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.37.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13819230 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-windows-amd64.zip.1’

ngrok-stable-window 100%[===================>]  13.18M  18.6MB/s    in 0.7s    

2021-03-18 08:07:21 (18.6 MB/s) - ‘ngrok-stable-windows-amd64.zip.1’ saved [13819230/13819230]

Archive:  ngrok-stable-windows-amd64.zip
replace ngrok.exe? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok.exe               
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
# from mtcnn.mtcnn import MTCNN
from numpy import load
from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
import pickle
# from trainingfile import get_embedding
from funcutils import *
# from trainingfile import get_embedding

In [ ]:
#Run This cell to train the network
# !python trainingfile.py

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://a11fgdip0m6-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from PIL import Image
import pickle
def extract_face(filename, required_size=(160, 160)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
#NOW TESTING FOR AN IMAGE
new_facenet_model_for_getting_embedding = load_model('/content/drive/MyDrive/MiniProject/facenet_keras.h5')
def predict_image(filepath):
  f=open('/content/drive/MyDrive/MiniProject/picklefile.txt','rb')
  d=pickle.loads(f.read())
  model=d['model']
  out_encoder=d['outencoder']
  X=list()
  faces=list()
  testembed=list()
  face=extract_face(filepath)
  #face.shape
  faces.append(face)
  X.extend(faces)
  Predic=asarray(X)
  # load the facenet model
  print('Loaded Model')
  #Predic.shape
  for face_pixels in Predic:
    embedding = get_embedding(new_facenet_model_for_getting_embedding , face_pixels)
    testembed.append(embedding)
  testembed = asarray(testembed)
  print(testembed.shape)
  # prediction for the face
  #samples = expand_dims(testembed, axis=0)
  #print(samples.shape)
  yhat_class = model.predict(testembed)
  yhat_prob = model.predict_proba(testembed)
  # get name
  class_index = yhat_class[0]
  class_probability = yhat_prob[0,class_index] * 100
  print(yhat_prob,int(yhat_prob[0,class_index]))
  return out_encoder.inverse_transform(yhat_class)[0]
  


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,flash,request,redirect,url_for,render_template
from werkzeug.utils import secure_filename
import os
import os
import socket
import time
from pyngrok import ngrok

host = '1.tcp.ngrok.io'
port = 12340

# Create a TCP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind a local socket to the port
server_address = ("", port)
sock.bind(server_address)
sock.listen(1)

# Open a ngrok tunnel to the socket

app = Flask(__name__,template_folder='/content')
# run_with_ngrok(app)   #starts ngrok when the app is run
public_url = ngrok.connect(port, "tcp", options={"remote_addr": "{}:{}".format(host, port)})
print("ngrok tunnel \"{}\" -> \"tcp://127.0.0.1:{}/\"".format(public_url, port))

@app.route("/",methods=['GET','POST'])
def uploadfile():
  if request.method=='POST':
    file=request.files['image']
    if file:
      filename=secure_filename(file.filename)
      file.save(filename)
      print('image saved')
      person_name=predict_image(filepath=f'/content/{filename}')
      while True:
          connection = None
          try:
              # Wait for a connection
              print("\nWaiting for a connection ...")
              connection, client_address = sock.accept()

              print("... connection established from {}".format(client_address))
              # name=predict_image(filepath=path)
              # Receive the message, send a response
              while True:
                  data = connection.recv(1024)
                  if data:
                      print("Received: {}".format(data.decode("utf-8")))

                      message = person_name
                      print("Sending: {}".format(message))
                      connection.sendall(message.encode("utf-8"))
                      sock.close()
                      return person_name
                  else:
                      break
          except KeyboardInterrupt:
              print(" Shutting down server.")

              if connection:
                  connection.close()
              break
      sock.close()
      return person_name
      # return "got your image"
  else:
    return render_template('sample.html')
@app.route('/home')
def index():
  return "<h1>this is not home page</h1>"
app.run()

ngrok tunnel "NgrokTunnel: "tcp://0.tcp.ngrok.io:11000" -> "localhost:12340"" -> "tcp://127.0.0.1:12340/"
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2021 08:16:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2021 08:16:12] "GET /favicon.ico HTTP/1.1" 404 -


image saved
Loaded Model
(1, 128)
[[9.99999783e-01 3.82940925e-08 1.01882904e-07 2.51464822e-08
  5.20559453e-08]] 0

Waiting for a connection ...


127.0.0.1 - - [18/Mar/2021 08:16:32] "POST / HTTP/1.1" 200 -


... connection established from ('127.0.0.1', 56698)
Received: pls send data
Sending: ben_afflek


t=2021-03-18T08:16:32+0000 lvl=warn msg="failed to open private leg" id=e54ea33d39e3 typ=proxy privaddr=localhost:12340 err="dial tcp 127.0.0.1:12340: connect: connection refused"
127.0.0.1 - - [18/Mar/2021 08:16:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2021 08:16:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2021 08:16:52] "GET /favicon.ico HTTP/1.1" 404 -
